In [ ]:
# Встановлення необхідних бібліотек
!pip install -U datasets transformers torch sentence-transformers qdrant-client langchain langchain-community langchain-qdrant ragas google-generativeai vllm accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.

In [ ]:
# Імпорт всіх необхідних бібліотек
import os
import json
import time
import random
from typing import List, Dict, Any
import numpy as np
import pandas as pd
from tqdm import tqdm

# Datasets та ML
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

# Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# LangChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

# Google Gemini для LLM as a Judge
import google.generativeai as genai

# RAGAS для оцінки
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_recall, context_precision

# Налаштування GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Використовується пристрій: {device}")

Використовується пристрій: cuda


In [ ]:
# Виправлена функція підготовки даних для NewsQA
def prepare_test_data_newsqa_correct(dataset, num_samples=2000):
    """Правильна підготовка тестових питань та відповідей з датасету NewsQA"""
    test_data = []
    contexts_for_kb = []

    for i, item in enumerate(dataset):
        if len(test_data) >= num_samples:
            break

        # Отримуємо параграф (контекст)
        paragraph = item.get('paragraph', '').strip()

        # Отримуємо списки питань та відповідей
        questions = item.get('questions', [])
        answers = item.get('answers', [])

        # Перевіряємо, що є дані і списки однакової довжини
        if paragraph and questions and answers and len(questions) == len(answers):
            # Проходимо по всіх парах питання-відповідь для цього параграфу
            for question, answer in zip(questions, answers):
                question = question.strip()
                answer = answer.strip()

                # Фільтруємо за якістю
                if (len(question) >= 5 and len(answer) >= 2 and len(paragraph) >= 100 and
                    len(answer) <= 300 and len(paragraph) <= 8000):

                    # Перевіряємо, що відповідь пов'язана з контекстом
                    # (для NewsQA це може бути не завжди дослівне входження)
                    test_data.append({
                        'question': question,
                        'answer': answer,
                        'context': paragraph
                    })

                    # Додаємо контекст до бази знань (без дублікатів)
                    if paragraph not in contexts_for_kb:
                        contexts_for_kb.append(paragraph)

    return test_data, contexts_for_kb

# Завантажуємо датасет та підготовлюємо дані
print("Завантаження NewsQA датасету...")
dataset = load_dataset("StellarMilk/newsqa", split="train[:2000]")

test_data, all_contexts = prepare_test_data_newsqa_correct(dataset)
print(f"Підготовлено {len(test_data)} тестових прикладів")
print(f"Зібрано {len(all_contexts)} контекстів для бази знань")

# Показуємо приклад
if test_data:
    print("\nПриклад тестових даних:")
    print(f"Питання: {test_data[0]['question']}")
    print(f"Відповідь: {test_data[0]['answer']}")
    print(f"Контекст: {test_data[0]['context'][:300]}...")

    print(f"\nЩе кілька прикладів питань:")
    for i in range(min(5, len(test_data))):
        print(f"{i+1}. Q: {test_data[i]['question']}")
        print(f"   A: {test_data[i]['answer']}")
        print()

Завантаження NewsQA датасету...
Підготовлено 2005 тестових прикладів
Зібрано 309 контекстів для бази знань

Приклад тестових даних:
Питання: Who is Ross Lovegrove?
Відповідь: Designer
Контекст: "Everything can be improved." -- Ross Lovegrove



Designer Ross Lovegrove



Fiercely original and unapologetically innovative, Ross Lovegrove describes himself, somewhat reluctantly, as an industrial designer. "It's not as grimy, it's not as deep and dark as that sounds," he says. Famous for his t...

Ще кілька прикладів питань:
1. Q: Who is Ross Lovegrove?
   A: Designer

2. Q: Where is his work held?
   A: in permanent collections of various design museums around the world, including the Museum of Modern Art in New York (MOMA), the Design Museum in London and the Vitra Design Museum Weil Am Rhein, Basel, Switzerland.

3. Q: Where is  lovegrove's work held?
   A: permanent collections of various design museums around the world,

4. Q: What is the name of the designer?
   A: Ross Lovegrove

5.

In [ ]:
# Додаткова фільтрація та покращення якості даних
def clean_newsqa_data_improved(test_data, all_contexts):
    """Покращена очистка даних NewsQA"""
    cleaned_test_data = []
    cleaned_contexts = list(set(all_contexts))  # Видаляємо дублікати контекстів

    # Статистика для фільтрації
    question_lengths = [len(item['question']) for item in test_data]
    answer_lengths = [len(item['answer']) for item in test_data]

    q_min, q_max = np.percentile(question_lengths, [10, 90])
    a_min, a_max = np.percentile(answer_lengths, [5, 95])

    print(f"Статистика довжин:")
    print(f"Питання: {q_min:.0f} - {q_max:.0f} символів")
    print(f"Відповіді: {a_min:.0f} - {a_max:.0f} символів")

    for item in test_data:
        question = item['question'].strip()
        answer = item['answer'].strip()
        context = item['context'].strip()

        # Розширені критерії фільтрації
        if (q_min <= len(question) <= q_max and
            a_min <= len(answer) <= a_max and
            len(context) >= 200 and
            not question.endswith('?') or question.endswith('?')):  # Приймаємо з ? і без

            # Додаткові перевірки якості
            if (not answer.lower().startswith(('the answer is', 'according to')) and
                len(answer.split()) >= 1 and len(answer.split()) <= 50):

                cleaned_test_data.append({
                    'question': question,
                    'answer': answer,
                    'context': context
                })

    return cleaned_test_data, cleaned_contexts

# Очищуємо дані
test_data, all_contexts = clean_newsqa_data_improved(test_data, all_contexts)
print(f"\nПісля покращеної очистки:")
print(f"Тестових прикладів: {len(test_data)}")
print(f"Унікальних контекстів: {len(all_contexts)}")

# Показуємо фінальні приклади
if test_data:
    print("\nФінальні приклади:")
    for i in range(min(3, len(test_data))):
        print(f"\n--- Приклад {i+1} ---")
        print(f"Питання: {test_data[i]['question']}")
        print(f"Відповідь: {test_data[i]['answer']}")
        print(f"Контекст: {test_data[i]['context'][:200]}...")

Статистика довжин:
Питання: 22 - 56 символів
Відповіді: 3 - 74 символів

Після покращеної очистки:
Тестових прикладів: 1928
Унікальних контекстів: 309

Фінальні приклади:

--- Приклад 1 ---
Питання: Who is Ross Lovegrove?
Відповідь: Designer
Контекст: "Everything can be improved." -- Ross Lovegrove



Designer Ross Lovegrove



Fiercely original and unapologetically innovative, Ross Lovegrove describes himself, somewhat reluctantly, as an industria...

--- Приклад 2 ---
Питання: Where is his work held?
Відповідь: in permanent collections of various design museums around the world, including the Museum of Modern Art in New York (MOMA), the Design Museum in London and the Vitra Design Museum Weil Am Rhein, Basel, Switzerland.
Контекст: "Everything can be improved." -- Ross Lovegrove



Designer Ross Lovegrove



Fiercely original and unapologetically innovative, Ross Lovegrove describes himself, somewhat reluctantly, as an industria...

--- Приклад 3 ---
Питання: Where is  lovegrove's wo

In [ ]:
from google.colab import userdata
GEMINI_KEY=userdata.get('GEMINI_KEY')

In [ ]:
# Налаштування Google Gemini API
genai.configure(api_key=GEMINI_KEY)

def generate_answer_with_small_llm(question: str) -> str:
    """Генерація відповіді без контексту використовуючи малу модель через Gemini"""
    try:
        model = genai.GenerativeModel('gemini-2.0-flash')

        prompt = f"""You are a helpful assistant. Answer the following question concisely and accurately based on your knowledge.

Question: {question}

Answer:"""

        response = model.generate_content(prompt)
        time.sleep(5)  # Пауза між запитами
        return response.text.strip()
    except Exception as e:
        print(f"Помилка при генерації відповіді: {e}")
        return "Не вдалося згенерувати відповідь"

# Генеруємо відповіді без RAG для тестового набору
print("Генерація відповідей без RAG...")
no_rag_answers = []

for i, item in enumerate(tqdm(test_data[:50])):  # Беремо менше для економії API запитів
    question = item['question']
    generated_answer = generate_answer_with_small_llm(question)
    no_rag_answers.append({
        'question': question,
        'true_answer': item['answer'],
        'generated_answer': generated_answer,
        'context': item['context']
    })

print(f"Згенеровано {len(no_rag_answers)} відповідей без RAG")

Генерація відповідей без RAG...


100%|██████████| 50/50 [04:58<00:00,  5.98s/it]

Згенеровано 50 відповідей без RAG


In [ ]:
no_rag_answers[0]

{'question': 'Who is Ross Lovegrove?',
 'true_answer': 'Designer',
 'generated_answer': 'Ross Lovegrove is a Welsh designer and visionary known for his organic and innovative approach to design, often incorporating biomimicry and advanced materials. He is known for his work across various fields, including furniture, lighting, transportation, and product design.',
 'context': '"Everything can be improved." -- Ross Lovegrove\n\n\n\nDesigner Ross Lovegrove\n\n\n\nFiercely original and unapologetically innovative, Ross Lovegrove describes himself, somewhat reluctantly, as an industrial designer. "It\'s not as grimy, it\'s not as deep and dark as that sounds," he says. Famous for his tactile and sensual fluid forms, he takes his inspiration from nature to create an organic minimalism that he calls "fat free" design.\n\n\n\n"Nature is a very big part of my work and always has been. I\'ve never seen it as a trend or a fashion," he told CNN.\n\n\n\nIn the early 1980s, with Frog Design in West

In [ ]:
# Ініціалізація Qdrant клієнта (in-memory)
qdrant_client = QdrantClient(":memory:")

# Налаштування embedding моделі
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embedding_dim = 384

print(f"Розмірність вбудовувань: {embedding_dim}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Розмірність вбудовувань: 384


In [ ]:
# Розбиття текстів на частини
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

# Розбиваємо всі контексти на чанки
print("Розбиття текстів на частini...")
all_chunks = []
chunk_metadata = []

for i, context in enumerate(tqdm(all_contexts)):
    chunks = text_splitter.split_text(context)
    for j, chunk in enumerate(chunks):
        if len(chunk.strip()) > 50:  # Фільтруємо занадто короткі чанки
            all_chunks.append(chunk)
            chunk_metadata.append({
                'source_id': i,
                'chunk_id': j,
                'text': chunk
            })

print(f"Створено {len(all_chunks)} текстових чанків")

Розбиття текстів на частini...


100%|██████████| 309/309 [00:00<00:00, 11085.13it/s]

Створено 2581 текстових чанків


In [ ]:
# Створення колекції у Qdrant
collection_name = "knowledge_base"

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

print(f"Створено колекцію '{collection_name}' у Qdrant")

Створено колекцію 'knowledge_base' у Qdrant


In [ ]:
# Генерація вбудовувань та збереження у векторному сховищі
print("Генерація вбудовувань та індексація...")
batch_size = 32

for i in tqdm(range(0, len(all_chunks), batch_size)):
    batch_chunks = all_chunks[i:i+batch_size]
    batch_metadata = chunk_metadata[i:i+batch_size]

    # Генеруємо вбудовування
    embeddings = embedding_model.encode(batch_chunks)

    # Створюємо точки для Qdrant
    points = []
    for j, (chunk, metadata, embedding) in enumerate(zip(batch_chunks, batch_metadata, embeddings)):
        points.append(PointStruct(
            id=i + j,
            vector=embedding.tolist(),
            payload={
                'text': chunk,
                'source_id': metadata['source_id'],
                'chunk_id': metadata['chunk_id']
            }
        ))

    # Зберігаємо у Qdrant
    qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )

print("Векторне сховище створено успішно!")

Генерація вбудовувань та індексація...


100%|██████████| 81/81 [00:05<00:00, 14.33it/s]

Векторне сховище створено успішно!


In [ ]:
def calculate_recall_at_k(test_data, k_values=[5, 10, 20, 30]):
    """Розрахунок Recall@K для retrieval моделі"""
    recall_scores = {k: [] for k in k_values}

    print("Обчислення Recall@K...")

    for item in tqdm(test_data[:50]):  # Беремо підмножину для швидкості
        question = item['question']
        true_context = item['context']

        # Генеруємо вбудовування для запиту
        query_embedding = embedding_model.encode([question])[0]

        # Пошук у векторному сховищі
        for k in k_values:
            search_results = qdrant_client.search(
                collection_name=collection_name,
                query_vector=query_embedding.tolist(),
                limit=k
            )

            # Перевіряємо, чи знайдений релевантний контекст
            retrieved_texts = [result.payload['text'] for result in search_results]

            # Спрощена перевірка релевантності (перевіряємо перетин слів)
            relevant_found = False
            true_words = set(true_context.lower().split())

            for retrieved_text in retrieved_texts:
                retrieved_words = set(retrieved_text.lower().split())
                # Якщо перетин слів більше 50%, вважаємо релевантним
                if len(true_words.intersection(retrieved_words)) / len(true_words.union(retrieved_words)) > 0.3:
                    relevant_found = True
                    break

            recall_scores[k].append(1 if relevant_found else 0)

    # Обчислюємо середні значення
    avg_recall = {}
    for k in k_values:
        avg_recall[k] = np.mean(recall_scores[k])
        print(f"Recall@{k}: {avg_recall[k]:.3f}")

    return avg_recall

recall_results = calculate_recall_at_k(test_data)

Обчислення Recall@K...


  0%|          | 0/50 [00:00<?, ?it/s]<ipython-input-23-fc341a8f9765>:16: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(
100%|██████████| 50/50 [00:01<00:00, 33.39it/s]

Recall@5: 0.060
Recall@10: 0.060
Recall@20: 0.060
Recall@30: 0.060


In [ ]:
def retrieve_relevant_contexts(question: str, k: int = 10) -> List[str]:
    """Пошук релевантних контекстів для питання"""
    query_embedding = embedding_model.encode([question])[0]

    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=k
    )

    return [result.payload['text'] for result in search_results]

def generate_rag_answer(question: str, contexts: List[str]) -> str:
    """Генерація відповіді з використанням RAG"""
    try:
        model = genai.GenerativeModel('gemini-2.0-flash-exp')

        # Об'єднуємо контексти
        context_text = "\n\n".join(contexts)

        prompt = f"""Based on the following context, answer the question concisely and accurately.

Context:
{context_text}

Question: {question}

Answer:"""

        response = model.generate_content(prompt)
        time.sleep(5)  # Пауза між запитами
        return response.text.strip()
    except Exception as e:
        print(f"Помилка при генерації RAG відповіді: {e}")
        return "Не вдалося згенерувати відповідь"

# Генерація відповідей з RAG
print("Генерація відповідей з RAG...")
rag_answers = []

for item in tqdm(test_data[:50]):  # Беремо ту ж кількість для порівняння
    question = item['question']

    # Отримуємо релевантні контексти
    relevant_contexts = retrieve_relevant_contexts(question, k=10)

    # Генеруємо відповідь з RAG
    rag_answer = generate_rag_answer(question, relevant_contexts)

    rag_answers.append({
        'question': question,
        'true_answer': item['answer'],
        'generated_answer': rag_answer,
        'contexts': relevant_contexts,
        'original_context': item['context']
    })

print(f"Згенеровано {len(rag_answers)} відповідей з RAG")

Генерація відповідей з RAG...


  0%|          | 0/50 [00:00<?, ?it/s]<ipython-input-24-d1b35bc52af2>:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(
100%|██████████| 50/50 [04:47<00:00,  5.75s/it]

Згенеровано 50 відповідей з RAG


In [ ]:
rag_answers[1]

{'question': 'Where is his work held?',
 'true_answer': 'in permanent collections of various design museums around the world, including the Museum of Modern Art in New York (MOMA), the Design Museum in London and the Vitra Design Museum Weil Am Rhein, Basel, Switzerland.',
 'generated_answer': 'His work is held in permanent collections of various design museums around the world, including the Museum of Modern Art in New York (MOMA), the Design Museum in London and the Vitra Design Museum Weil Am Rhein, Basel, Switzerland.',
 'contexts': ["The suit is being brought by the Authors' Guild, its equivalents in Australia, Quebec, and the UK, and a large group of individual authors. Its target: some major US universities, including Michigan, the University of California system, and Cornell.",
  "His friends were able to remove him from the rubble of the New Victorian School at 11 a.m. the day after the earthquake, about 18 hours later. Later that week, he was airlifted out of Port-au-Prince b

In [ ]:
def evaluate_answer_quality(question: str, true_answer: str, generated_answer: str) -> Dict[str, Any]:
    """Оцінка якості відповіді за допомогою LLM as a Judge"""
    try:
        model = genai.GenerativeModel('gemini-2.5-flash-preview-05-20')

        prompt = f"""You are an expert evaluator. Please evaluate the quality of the generated answer compared to the true answer.

Question: {question}
True Answer: {true_answer}
Generated Answer: {generated_answer}

Please provide:
1. A binary score (0 or 1) where 1 means the generated answer is correct/relevant and 0 means incorrect/irrelevant
2. A detailed score from 1-5 where:
   - 1: Completely incorrect or irrelevant
   - 2: Mostly incorrect with some relevant information
   - 3: Partially correct
   - 4: Mostly correct with minor issues
   - 5: Completely correct and comprehensive

Format your response as:
Binary Score: [0 or 1]
Detailed Score: [1-5]
Explanation: [Brief explanation of your scoring]"""

        response = model.generate_content(prompt)
        time.sleep(5)

        # Парсимо відповідь
        response_text = response.text
        lines = response_text.split('\n')

        binary_score = 0
        detailed_score = 1
        explanation = ""

        for line in lines:
            if 'Binary Score:' in line:
                try:
                    binary_score = int(line.split(':')[1].strip())
                except:
                    binary_score = 0
            elif 'Detailed Score:' in line:
                try:
                    detailed_score = int(line.split(':')[1].strip())
                except:
                    detailed_score = 1
            elif 'Explanation:' in line:
                explanation = line.split(':', 1)[1].strip()

        return {
            'binary_score': binary_score,
            'detailed_score': detailed_score,
            'explanation': explanation
        }

    except Exception as e:
        print(f"Помилка при оцінці: {e}")
        return {
            'binary_score': 0,
            'detailed_score': 1,
            'explanation': "Помилка оцінки"
        }

# Оцінка відповідей без RAG
print("Оцінка якості відповідей без RAG...")
no_rag_evaluations = []

for item in tqdm(no_rag_answers):
    evaluation = evaluate_answer_quality(
        item['question'],
        item['true_answer'],
        item['generated_answer']
    )
    no_rag_evaluations.append(evaluation)

# Оцінка відповідей з RAG
print("Оцінка якості відповідей з RAG...")
rag_evaluations = []

for item in tqdm(rag_answers):
    evaluation = evaluate_answer_quality(
        item['question'],
        item['true_answer'],
        item['generated_answer']
    )
    rag_evaluations.append(evaluation)

Оцінка якості відповідей без RAG...


100%|██████████| 50/50 [09:36<00:00, 11.53s/it]


Оцінка якості відповідей з RAG...


100%|██████████| 50/50 [07:31<00:00,  9.03s/it]


In [ ]:
# Обчислення метрик
def calculate_metrics(evaluations):
    """Обчислення середніх метрик"""
    binary_scores = [eval_item['binary_score'] for eval_item in evaluations]
    detailed_scores = [eval_item['detailed_score'] for eval_item in evaluations]

    return {
        'accuracy': np.mean(binary_scores),
        'avg_detailed_score': np.mean(detailed_scores),
        'total_samples': len(evaluations)
    }

no_rag_metrics = calculate_metrics(no_rag_evaluations)
rag_metrics = calculate_metrics(rag_evaluations)

print("=== РЕЗУЛЬТАТИ ЕКСПЕРИМЕНТУ ===")
print()
print("1. Якість Retrieval моделі (Recall@K):")
for k, score in recall_results.items():
    print(f"   Recall@{k}: {score:.3f}")

print()
print("2. Якість відповідей без RAG:")
print(f"   Accuracy (binary): {no_rag_metrics['accuracy']:.3f}")
print(f"   Avg detailed score: {no_rag_metrics['avg_detailed_score']:.3f}")
print(f"   Total samples: {no_rag_metrics['total_samples']}")

print()
print("3. Якість відповідей з RAG:")
print(f"   Accuracy (binary): {rag_metrics['accuracy']:.3f}")
print(f"   Avg detailed score: {rag_metrics['avg_detailed_score']:.3f}")
print(f"   Total samples: {rag_metrics['total_samples']}")

print()
print("4. Порівняння RAG vs No-RAG:")
accuracy_improvement = rag_metrics['accuracy'] - no_rag_metrics['accuracy']
detailed_improvement = rag_metrics['avg_detailed_score'] - no_rag_metrics['avg_detailed_score']

print(f"   Покращення accuracy: {accuracy_improvement:+.3f}")
print(f"   Покращення detailed score: {detailed_improvement:+.3f}")

if accuracy_improvement > 0:
    print("   ✅ RAG показав кращі результати за accuracy")
else:
    print("   ❌ RAG не покращив accuracy")

if detailed_improvement > 0:
    print("   ✅ RAG показав кращі результати за детальною оцінкою")
else:
    print("   ❌ RAG не покращив детальну оцінку")

=== РЕЗУЛЬТАТИ ЕКСПЕРИМЕНТУ ===

1. Якість Retrieval моделі (Recall@K):
   Recall@5: 0.060
   Recall@10: 0.060
   Recall@20: 0.060
   Recall@30: 0.060

2. Якість відповідей без RAG:
   Accuracy (binary): 0.380
   Avg detailed score: 2.700
   Total samples: 50

3. Якість відповідей з RAG:
   Accuracy (binary): 0.660
   Avg detailed score: 3.540
   Total samples: 50

4. Порівняння RAG vs No-RAG:
   Покращення accuracy: +0.280
   Покращення detailed score: +0.840
   ✅ RAG показав кращі результати за accuracy
   ✅ RAG показав кращі результати за детальною оцінкою


In [ ]:
# Показуємо кілька прикладів порівняння
print("=== ПРИКЛАДИ ВІДПОВІДЕЙ ===")

for i in range(min(5, len(no_rag_answers))):
    print(f"\n--- Приклад {i+1} ---")
    print(f"Питання: {no_rag_answers[i]['question']}")
    print(f"Правильна відповідь: {no_rag_answers[i]['true_answer']}")
    print(f"Без RAG: {no_rag_answers[i]['generated_answer']}")
    print(f"З RAG: {rag_answers[i]['generated_answer']}")
    print(f"Оцінка без RAG: Binary={no_rag_evaluations[i]['binary_score']}, Detailed={no_rag_evaluations[i]['detailed_score']}")
    print(f"Оцінка з RAG: Binary={rag_evaluations[i]['binary_score']}, Detailed={rag_evaluations[i]['detailed_score']}")

=== ПРИКЛАДИ ВІДПОВІДЕЙ ===

--- Приклад 1 ---
Питання: Who is Ross Lovegrove?
Правильна відповідь: Designer
Без RAG: Ross Lovegrove is a Welsh designer and visionary known for his organic and innovative approach to design, often incorporating biomimicry and advanced materials. He is known for his work across various fields, including furniture, lighting, transportation, and product design.
З RAG: Ross Lovegrove is an industrial designer known for his tactile and sensual fluid forms, taking inspiration from nature to create an organic minimalism.
Оцінка без RAG: Binary=1, Detailed=5
Оцінка з RAG: Binary=1, Detailed=5

--- Приклад 2 ---
Питання: Where is his work held?
Правильна відповідь: in permanent collections of various design museums around the world, including the Museum of Modern Art in New York (MOMA), the Design Museum in London and the Vitra Design Museum Weil Am Rhein, Basel, Switzerland.
Без RAG: To answer accurately, I need to know who "he" refers to. Please provide more c